<a href="https://colab.research.google.com/github/kop57890/acerml/blob/main/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/acerml/raw/main/poem_train.txt"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/acerml/raw/main/poem_test.txt"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
u = train_df["作者"].unique()
write_index = {name:i for i, name in enumerate(u)}
index_writer = {v:k for k, v in write_index.items()}
y_train = train_df["作者"].replace(write_index)
y_test = test_df["作者"].replace(write_index)
y_test

In [ ]:
import jieba
p = train_df["內容"][0]
" ".join(jieba.cut(p))
def poemcut(p):
  return " ".join(jieba.cut(p))
x_train = train_df["內容"].apply(poemcut)
x_test = test_df["內容"].apply(poemcut)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)
x_train_count

<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8

In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, pre),
      index = [name + "(True)" for name in u],
      columns = [name + "(Pred)" for name in u]
      )

,李白(Pred),杜甫(Pred),白居易(Pred)
李白(True),8,1,1
杜甫(True),1,8,1
白居易(True),1,1,8


In [ ]:
p = input("輸入詩:")
test = vec.transform([poemcut(p)])
prob = clf.predict_proba(test)[0]
for name, proba in zip(u, prob):
  print(name, "的機率", proba)

輸入詩:君不見 黃河之水天上來
李白 的機率 0.8573778616749936
杜甫 的機率 0.1068457653457308
白居易 的機率 0.035776372979274405
